In [3]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.special

In [4]:
%matplotlib inline

# Neural Network

A weights matrix for passing inputs forward into a neural network is presented below. The weights matrix will be transposed (.T) when a net is passing backprop.

<br>w11  w21 w31 w41 
<br>w12
<br>w13
<br>w14   &nbsp; &nbsp; &nbsp; &nbsp;&nbsp; &nbsp; &nbsp;       wnn    


In [31]:
class Neural_network:
    def __init__(self, in_nodes, hidden_nodes, out_nodes, lr=0.3):
        self.in_nodes = in_nodes
        self.h_nodes = hidden_nodes
        self.out_nodes = out_nodes
        
        self.lr = lr
        self.activation_function = lambda x: scipy.special.expit(x)
        
        # weights matrix with values from -0.5 to 0.5
        self.w_input_hidden = np.random.rand(self.h_nodes, self.in_nodes) - 0.5
        self.w_hidden_output = np.random.rand(self.out_nodes, self.h_nodes) - 0.5
        
    def train(self, inputs_list, targets_list):
        inputs = np.array(inputs_list, ndmin=2).T
        targets = np.array(targets_list, ndmin=2).T
        
        final_outputs, hidden_outputs = self.out(inputs_list)
        
        
        output_errors = targets - final_outputs
        hidden_errors = np.dot(self.w_hidden_output.T, output_errors)
        
        
        self.w_hidden_output += self.lr * np.dot((output_errors * final_outputs * (1 - final_outputs)), 
                                                 (hidden_outputs.T))
        
        self.w_input_hidden += self.lr * np.dot((hidden_errors * hidden_outputs  * (1 - hidden_outputs)), 
                                                 (inputs.T))
    
    def out(self, inputs_list):
        inputs = np.array(inputs_list, ndmin=2).T
        
        hidden_inputs = np.dot(self.w_input_hidden, inputs)
        hidden_outputs = self.activation_function(hidden_inputs)
        
        final_inputs = np.dot(self.w_hidden_output, hidden_outputs)
        final_outputs = self.activation_function(final_inputs)
        
        return final_outputs, hidden_outputs
    
    
    def get_predict(self, inputs_list):
        final_outputs, hidden_outputs = self.out(inputs_list)
        return final_outputs
    
    
# A heuristic for improving the weights inizialization
# self.wih = np.random.normal(0.0, pow(self.h_nodes, -0.5), (self.h_nodes, self.in_nodes))
# self.who = np.random.normal(0.0, pow(self.out_nodes, -0.5), (self.out_nodes, self.h_nodes))


In [32]:
input_nodes = 784
hidden_nodes = 100
output_nodes = 10

lr = 0.3
net = Neural_network(input_nodes, hidden_nodes, output_nodes, lr)

In [33]:
with open('./mnist_train.csv', 'r') as f:
    training_data_list = f.readlines()

In [ ]:
epochs = 1
for epoch in range(epochs):
    for ind_rec, record in enumerate(training_data_list[1:]):
        print(f'epoch {epoch}, record: {ind_rec}/{len(training_data_list)}', end='\r')
        
        all_values = record.split(',')
        inputs = np.asfarray(all_values[1:]) / 255 * 0.99 + 0.01
        targets = np.zeros(output_nodes) + 0.01
        targets[int(all_values[0])] = 0.99

        net.train(inputs, targets)

In [ ]:
with open('./mnist_test.csv', 'r') as f:
    test_data_list = f.readlines()[1:]

In [ ]:
scorecard = []

for record in test_data_list:
    all_values = record.split(',')
    correct_label = int(all_values[0])
    
    inputs = np.asfarray(all_values[1:]) / 255 * 0.99 + 0.01
    outputs = net.get_predict(inputs)
    out_label = np.argmax(outputs)
    
    # print(f'The net answer is: {out_label}, The correct answer is: {correct_label}')
    
    if out_label == correct_label:
        scorecard.append(1)
    else:
        scorecard.append(0)
        
        
print(f'\nAccuracy of the network is {scorecard.count(1) / len(scorecard)}')